# 4 aggregate to 12 level

* ## Load the data and analysis libraries

> ### Load libraries

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm as tqdm


In [ ]:
IS_LOCAL_DEMO = True
# IS_LOCAL_DEMO = False

> ### All models

In [ ]:
model_name = "knn"
# model_name = "ridge"
# model_name = "rf"
# model_name = "xgb"
# model_name = "lgm"
# model_name = "mlp"

In [ ]:
if(IS_LOCAL_DEMO):
    sub=pd.read_csv("submission_{}.csv.gz".format(model_name)) # for local demo

else:
    sub=pd.read_csv("submission_{}.csv".format(model_name)) # for kaggle


In [ ]:
sub.head(3)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,2.403333,1.030000,1.030000,0.343333,1.030000,0.343333,5.493333,4.463333,1.716667,...,0.343333,1.030000,0.000000,1.373333,1.030000,1.030000,0.343333,1.030000,1.030000,0.343333
1,FOODS_1_001_CA_2_validation,0.343333,0.343333,0.343333,1.716667,0.343333,0.343333,1.716667,1.373333,0.343333,...,0.343333,0.343333,1.030000,0.686667,0.686667,1.030000,1.030000,0.686667,0.686667,1.030000
2,FOODS_1_001_CA_3_validation,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.000000,0.686667,0.343333,...,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667,0.686667


In [ ]:
train_sales = pd.read_csv('../datasets/sales_train_validation.csv')
submission_file = pd.read_csv('../datasets/sample_submission.csv')


* ## Aggregate for 12 level

> ### We refer to the M5 competitors Guide and the "Building aggregate time series" created by Sachin Agarwal to the aggregate format of the training result to fit for "sample submission."

[Building aggregate time series](https://www.kaggle.com/sachina/building-aggregate-time-series)

In [ ]:
sub_col = ['id']
sub_col.extend([f'd_{day}' for day in range(1914, 1914+28)])
sub.columns = sub_col

In [ ]:
train_sales = train_sales.merge(sub, on='id')

In [ ]:
total = ['Total']
train_sales['Total'] = 'Total'
train_sales['state_cat'] = train_sales.state_id + "_" + train_sales.cat_id
train_sales['state_dept'] = train_sales.state_id + "_" + train_sales.dept_id
train_sales['store_cat'] = train_sales.store_id + "_" + train_sales.cat_id
train_sales['store_dept'] = train_sales.store_id + "_" + train_sales.dept_id
train_sales['state_item'] = train_sales.state_id + "_" + train_sales.item_id
train_sales['item_store'] = train_sales.item_id + "_" + train_sales.store_id

In [ ]:
val_eval = ['validation', 'evaluation']

# creating lists for different aggregation levels
total = ['Total']
states = ['CA', 'TX', 'WI']
num_stores = [('CA',4), ('TX',3), ('WI',3)]
stores = [x[0] + "_" + str(y + 1) for x in num_stores for y in range(x[1])]
print(stores)  # store_id

['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']


In [ ]:
cats = ['FOODS', 'HOBBIES', 'HOUSEHOLD']
num_depts = [('FOODS',3), ('HOBBIES',2), ('HOUSEHOLD',2)]
depts = [x[0] + "_" + str(y + 1) for x in num_depts for y in range(x[1])]
state_cats = [state + "_" + cat for state in states for cat in cats]
state_depts = [state + "_" + dept for state in states for dept in depts]
store_cats = [store + "_" + cat for store in stores for cat in cats]
store_depts = [store + "_" + dept for store in stores for dept in depts]
print(state_cats)  # state and  category
print("=================================================================")
print(store_depts)  # store_id + state_id +category_id

['CA_FOODS', 'CA_HOBBIES', 'CA_HOUSEHOLD', 'TX_FOODS', 'TX_HOBBIES', 'TX_HOUSEHOLD', 'WI_FOODS', 'WI_HOBBIES', 'WI_HOUSEHOLD']
['CA_1_FOODS_1', 'CA_1_FOODS_2', 'CA_1_FOODS_3', 'CA_1_HOBBIES_1', 'CA_1_HOBBIES_2', 'CA_1_HOUSEHOLD_1', 'CA_1_HOUSEHOLD_2', 'CA_2_FOODS_1', 'CA_2_FOODS_2', 'CA_2_FOODS_3', 'CA_2_HOBBIES_1', 'CA_2_HOBBIES_2', 'CA_2_HOUSEHOLD_1', 'CA_2_HOUSEHOLD_2', 'CA_3_FOODS_1', 'CA_3_FOODS_2', 'CA_3_FOODS_3', 'CA_3_HOBBIES_1', 'CA_3_HOBBIES_2', 'CA_3_HOUSEHOLD_1', 'CA_3_HOUSEHOLD_2', 'CA_4_FOODS_1', 'CA_4_FOODS_2', 'CA_4_FOODS_3', 'CA_4_HOBBIES_1', 'CA_4_HOBBIES_2', 'CA_4_HOUSEHOLD_1', 'CA_4_HOUSEHOLD_2', 'TX_1_FOODS_1', 'TX_1_FOODS_2', 'TX_1_FOODS_3', 'TX_1_HOBBIES_1', 'TX_1_HOBBIES_2', 'TX_1_HOUSEHOLD_1', 'TX_1_HOUSEHOLD_2', 'TX_2_FOODS_1', 'TX_2_FOODS_2', 'TX_2_FOODS_3', 'TX_2_HOBBIES_1', 'TX_2_HOBBIES_2', 'TX_2_HOUSEHOLD_1', 'TX_2_HOUSEHOLD_2', 'TX_3_FOODS_1', 'TX_3_FOODS_2', 'TX_3_FOODS_3', 'TX_3_HOBBIES_1', 'TX_3_HOBBIES_2', 'TX_3_HOUSEHOLD_1', 'TX_3_HOUSEHOLD_2', 'WI_

In [ ]:
prods = list(train_sales.item_id.unique())  # product_list
prod_state = [prod + "_" + state for prod in prods for state in states]  # product_id+state_id
prod_store = [prod + "_" + store for prod in prods for store in stores]  # product_id + store_id

In [ ]:
quants = ['0.005', '0.025', '0.165', '0.250', '0.500', '0.750', '0.835', '0.975', '0.995']
days = range(1, 1913 + 29)
time_series_columns = [f'd_{i}' for i in days]

In [ ]:
def create_sales(name_list, group):
    sales = train_sales.groupby(group)[time_series_columns].sum() 
    return sales

In [ ]:
def create_quantile_dict(name_list = stores, group = 'store_id' ,X = False):
    sales = create_sales(name_list, group)
    sales = sales.iloc[:, 1857:] #using the last few months data only
    sales_quants = pd.DataFrame(index = sales.index)  
    for q in quants:
        sales_quants[q] = np.quantile(sales, float(q), axis = 1)  
    full_mean = pd.DataFrame(np.mean(sales, axis = 1))
    daily_means = pd.DataFrame(index = sales.index)  
    for i in range(7):
        daily_means[str(i)] = np.mean(sales.iloc[:, i::7], axis = 1) 
        
    daily_factors = daily_means / np.array(full_mean)
    
    daily_factors = pd.concat([daily_factors, daily_factors, daily_factors, daily_factors], axis = 1)
    daily_factors_np = np.array(daily_factors)

    factor_df = pd.DataFrame(daily_factors_np, columns = submission_file.columns[1:])
    factor_df.index = daily_factors.index

    for i,x in enumerate(tqdm(sales_quants.index)):
        for q in quants:
            v = sales_quants.loc[x, q] * np.array(factor_df.loc[x, :])
            if X:
                my_dict[x + "_X_" + q + "_validation"] = v
                my_dict[x + "_X_" + q + "_evaluation"] = v
            else:
                my_dict[x + "_" + q + "_validation"] = v
                my_dict[x + "_" + q + "_evaluation"] = v

In [ ]:
my_dict = {}
#adding prediction to my_dict on all 12 aggregation levels
create_quantile_dict(total, 'Total', X=True) #1
create_quantile_dict(states, 'state_id', X=True) #2
create_quantile_dict(stores, 'store_id', X=True) #3
create_quantile_dict(cats, 'cat_id', X=True) #4
create_quantile_dict(depts, 'dept_id', X=True) #5
create_quantile_dict(state_cats, 'state_cat') #6
create_quantile_dict(state_depts, 'state_dept') #7
create_quantile_dict(store_cats, 'store_cat') #8
create_quantile_dict(store_depts, 'store_dept') #9
create_quantile_dict(prods, 'item_id', X=True) #10
create_quantile_dict(prod_state, 'state_item') #11
create_quantile_dict(prod_store, 'item_store') #12

In [ ]:
pred_df = pd.DataFrame(my_dict)
pred_df = pred_df.transpose()
pred_df_reset = pred_df.reset_index()
final_pred = pd.merge(pd.DataFrame(submission_file.id), pred_df_reset, left_on = 'id', right_on = 'index')
del final_pred['index']
final_pred = final_pred.rename(columns={0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7', 7: 'F8', 8: 'F9',
                                        9: 'F10', 10: 'F11', 11: 'F12', 12: 'F13', 13: 'F14', 14: 'F15', 15: 'F16',
                                        16: 'F17', 17: 'F18', 18: 'F19', 19: 'F20', 20: 'F21', 21: 'F22', 
                                        22: 'F23', 23: 'F24', 24: 'F25', 25: 'F26', 26: 'F27', 27: 'F28'})

In [ ]:
final_pred = final_pred.fillna(0)

In [ ]:
final_pred.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,Total_X_0.005_validation,31458.870306,30294.597819,29471.972597,29740.698403,32735.252302,38475.552508,38687.746065,31458.870306,30294.597819,...,32735.252302,38475.552508,38687.746065,31458.870306,30294.597819,29471.972597,29740.698403,32735.252302,38475.552508,38687.746065
1,CA_X_0.005_validation,13773.411070,13036.507860,12642.805540,12651.854274,13796.412027,16709.000323,17215.188908,13773.411070,13036.507860,...,13796.412027,16709.000323,17215.188908,13773.411070,13036.507860,12642.805540,12651.854274,13796.412027,16709.000323,17215.188908
2,TX_X_0.005_validation,8240.036818,8015.861403,7755.424596,7913.512449,8414.425378,9501.853852,9882.815504,8240.036818,8015.861403,...,8414.425378,9501.853852,9882.815504,8240.036818,8015.861403,7755.424596,7913.512449,8414.425378,9501.853852,9882.815504
3,WI_X_0.005_validation,8553.242492,8369.797001,8218.915432,8309.787403,9541.533135,11124.206388,10498.643150,8553.242492,8369.797001,...,9541.533135,11124.206388,10498.643150,8553.242492,8369.797001,8218.915432,8309.787403,9541.533135,11124.206388,10498.643150
4,CA_1_X_0.005_validation,3207.951461,2945.859296,2875.196058,2908.712405,3258.446079,4033.782651,4102.102050,3207.951461,2945.859296,...,3258.446079,4033.782651,4102.102050,3207.951461,2945.859296,2875.196058,2908.712405,3258.446079,4033.782651,4102.102050


* ## Dataset persistence for submit

In [ ]:
if(IS_LOCAL_DEMO):
    final_pred.to_csv("submission_{}.csv.gz".format(model_name),compression='gzip',index=False)# for local demo
else:
    final_pred.to_csv("submission_{}.csv".format(model_name),index=False)# for kaggle

